# Processing thread for XMM-Newton EPIC observations

## Step 01

The first step does the following:

1. Checks if XMM-SAS is available
2. Sets the folders where the products will be downloaded and saved
3. Downloads full ODF (tar file) from the XMM archive
4. Untars the tar file and the internal .TAR file
5. Runs the cifbuild to create the CCF file
6. Runs odfingest

In [20]:
#
# imports
#
import os
import tarfile
import subprocess
import sys
import requests
import logging

In [21]:
# function to run shell process
# get the SAS version, will be used for the output folder 
def run_command(command):
    try:
        result = subprocess.run([command], shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
        retcode = result.returncode
        if retcode < 0:
            print("Warning: \"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
            logging.warning("\"{}\" was terminated by signal {} \n {}".format(command,-retcode,result.stdout.decode()))
        else:
            print("Info: \"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
            logging.info("\"{}\" returned {} \n {}".format(command,retcode,result.stdout.decode()))
    except OSError as e:
        print("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
        logging.error("Execution of \"{}\" failed:".format(command), e, file=sys.stderr)
    return retcode

In [22]:
#
# the OBS_ID to process
#
#obsid = "0721010501"
# PNLW with high background, rev 2524, mean_bkg=238 +/- 2869
obsid = "0721010401"
# PNLW with low background, rev 2522, mean_bkg=2.68 +/- 0.55

In [23]:
# environment
os.environ['SAS_CCFPATH'] = '/xdata/ccf/pub'
# check if XMM-SAS works
comm = "sasversion -v"
status = run_command(comm)
if (status != 0):
    print ("No XMM-SAS available to notebook! Please activate the XMM-SAS in console and then start notebook.")
    raise Exception
#
# some folders
#
home = os.path.expanduser('~')
wdir = f"{home}/IVAN/Cu-line"
if (not os.path.isdir(wdir)):
    print (f"The input working folder {wdir} does not exist.")
    raise FileNotFoundError
# the output directory, all files wil lbe saved there
outputDir = os.path.join(wdir,obsid)
if (not os.path.isdir(outputDir)):
    os.mkdir(outputDir)
#
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(levelname)s %(message)s',
                    filename=f'{wdir}/{obsid}_proc.log',
                    filemode='w')

Info: "sasversion -v" returned 0 
 sasversion (sasversion-1.3) [xmmsas_20180620_1732-17.0.0]



In [24]:
#
# The XMM archive access
#
url = f"http://nxsa.esac.esa.int/nxsa-sl/servlet/data-action-aio?obsno={obsid}&level=ODF"
#
#
tarFile = f"{outputDir}/{obsid}_ODF.tar"
#
if (not os.path.isfile(tarFile)):
    print (f"No ODF tar file for OBS_ID {obsid}, downloading from NXSA")
    logging.info ("No ODF tar file for OBS_ID {obsid}, downloading from NXSA")
    with requests.post(url) as r:
    #with requests.post(url, auth=('ivaltch', 'XXXX')) as r:
        r.raise_for_status() # ensure we notice bad responses
        with open(tarFile,"wb") as tmp:
            tmp.write(r.content)
else:
    print (f"ODF tar file {tarFile} found, will skip downloading it again")
    logging.info (f"ODF tar file {tarFile} found, will skip downloading it again")
#

No ODF tar file for OBS_ID 0721010401, downloading from NXSA


In [25]:
foundTar = False
os.chdir(outputDir)
#
# untar the file
#
with tarfile.open(tarFile,'r') as tar:
    for member in tar.getmembers():
        # check member is already in folder
        if (not os.path.isfile(f"{outputDir}/{member.name}")):
            print (f"Extracting {member.name}")
            f=tar.extract(member,path=outputDir)
        else:
            print (f"File {member.name} already extracted, skipping")
        if ("TAR" in member.name):
            foundTar = True
            print (f"Extracting the content of {member.name}")
            # will use the Linux tar, the one from python is problematic
            tarcomm = f"tar xf {member.name}"
            status = run_command(tarcomm)
            if (status != 0):
                raise Exception
            # remove the TAR file, we already extracted its content
            os.remove(f"{outputDir}/{member.name}")
if (not foundTar):
    raise FileNotFoundError
    logging.error("TAR file not found")


Extracting 2522_0721010401_SCX00000ATS.FIT
Extracting 2522_0721010401_SCX00000TCS.FIT
Extracting 2522_0721010401_SCX00000RAS.ASC
Extracting 2522_0721010401_SCX00000ROS.ASC
Extracting 2522_0721010401_SCX00000SUM.ASC
Extracting MANIFEST.564376
Extracting 2522_0721010401.TAR
Extracting the content of 2522_0721010401.TAR
Info: "tar xf 2522_0721010401.TAR" returned 0 
 


In [26]:
#
# now the SAS processing, must initialise HEADAS and XMMSAS before running the code!
#
odfdir = outputDir
os.chdir(odfdir)
os.environ["SAS_ODF"] = odfdir
ccffile = f"{odfdir}/ccf.cif"
logging.info(f"Set SAS_ODF to {odfdir}")
#
# cifbuild
print ("*** CIFBUILD")
comm = "cifbuild"
status = run_command(comm)
if (status != 0):
    raise Exception
#
# add the CCF file to environment
#
os.environ["SAS_CCF"] = ccffile
logging.info(f"Set SAS_CCF to {ccffile}")

*** CIFBUILD
Info: "cifbuild" returned 0 
 cifbuild:- Executing (routine): cifbuild calindexset=ccf.cif withccfpath=no usecanonicalname=no ccfpath=. recurse=no fileglob=*.ccf|*.CCF fullpath=no withobservationdate=no observationdate=now analysisdate=now category=XMMCCF ignorecategory=no masterindex=no withmasterindexset=no masterindexset=ccf.mif append=no  -w 1 -V 4
cifbuild:- cifbuild (cifbuild-4.9)  [xmmsas_20180620_1732-17.0.0] started:  2019-02-07T16:27:31.000
cifbuild:- Will ask the analysis date to the OAL.
cifbuild:- Using the ODF 2522_0721010401 found in /home/ivaltchanov/IVAN/Cu-line/0721010401
cifbuild:- Observation date: 2013-09-15T18:30:00.000
cifbuild:- Analysis date: 2019-02-07T16:27:31.000
cifbuild:- cifbuild (cifbuild-4.9)  [xmmsas_20180620_1732-17.0.0] ended:    2019-02-07T16:27:44.000



In [27]:
print ("*** ODFINGEST")
comm = "odfingest -V 1"
status = run_command(comm)
if (status != 0):
    raise Exception
print (f"*** all done for {obsid} and step #01")
logging.info (f"*** all done for {obsid} and step #01")

*** ODFINGEST
Info: "odfingest -V 1" returned 0 
 odfingest:- Executing (routine): odfingest odfdir=. withodfdir=no outdir=. summaryfile=0000_0000000000_SCX000SUM.SAS usecanonicalname=yes writepath=yes findinstrumentmodes=yes usehousekeeping=yes oalcheck=no  -w 1 -V 1
** odfingest: warning (TranslationNotAvailable), No look up possible [EPN Filter]It is not possible to translate FILTER. Its raw value is -nan
** odfingest: warning (HousekeepingParameterMissing), PN/U002 NoAvailableRows: No rows match the expression (F1119 == "STOPPED") && (#ROW >= 15090 && #ROW <= 15089) in /home/ivaltchanov/IVAN/Cu-line/0721010401/2522_0721010401_PNX00000PMH.FIT:PNPMH1
** odfingest: warning (SummaryOfWarnings), 
   warning HousekeepingParameterMissing silently occurred 7 times
   warning TranslationNotAvailable silently occurred 7 times

*** all done for 0721010401 and step #01
